In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
#from torch_geometric.loader import DataLoader 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GCNConv
import pandas as pd
import ast
import torch
from torch.nn import Module, Linear
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Linear
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.nn import Module, Linear
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import random
import math
import pickle
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib
import time
from torch_geometric.nn import GraphSAGE
import gc
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import time
import pickle
torch.cuda.empty_cache()
gc.collect()
matplotlib.use('Agg')

In [2]:
df = pd.read_csv('/home/scai/msr/aiy227513/scratch/research/superSetAnalysis.csv')
node_ids =[]
node_features = []
req_A=['citationVelocity', 'influentialCitationCount', 'isOpenAccess',
       'isPublisherLicence', 'numCitebBy', 'numCiting','year']
for index, row in df.iterrows():
    paperId = row['paperId']
    node_ids.append(paperId)
    f=[]
    for attribute in req_A:
            if attribute=="isOpenAccess":
             if row[attribute]==True:attribute_value=1
             else:attribute_value=0
            elif attribute=="isPublisherLicence":
              if row[attribute]==True:attribute_value=1
              else:attribute_value=0
            else:
               attribute_value = row[attribute]
            f.append(attribute_value)
    node_features.append(np.array(f).astype(np.float32))

In [3]:
df_test = pd.read_csv('/home/scai/msr/aiy227513/scratch/research/data/superSetAnalysis.csv').sample(frac=1)
node_ids_test =[]
node_features_test = []
sentence_test=[]
for index, row in df_test.iterrows():
    paperId = row['paperId']
    if paperId in node_ids:continue
    node_ids_test.append(paperId)
    f=[]
    sentence_test.append(row['abstract'])
    for attribute in req_A:
            if attribute=="isOpenAccess":
             if row[attribute]==True:attribute_value=1
             else:attribute_value=0
            elif attribute=="isPublisherLicence":
              if row[attribute]==True:attribute_value=1
              else:attribute_value=0
            else:
               attribute_value = row[attribute]
            f.append(attribute_value)
    node_features_test.append(np.array(f).astype(np.float32))

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
with open("/home/scai/msr/aiy227513/scratch/research/data/bert_encode.pkl", "rb") as file:
    embedding = pickle.load(file)
    file.close()
node_features = torch.tensor(node_features, dtype=torch.float)
embedding=torch.tensor(embedding, dtype=torch.float)
feature=torch.cat([node_features,embedding], dim=1).float()
feature =F.normalize(feature,p=2, dim=-1)

/var/tmp/pbs.3721100.pbshpc/ipykernel_191187/533348573.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  node_features = torch.tensor(node_features, dtype=torch.float)


Bert Embedding test data

In [6]:
class USE(nn.Module):
    def __init__(self, model_name='bert-base-multilingual-cased'):
        super(USE, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
    
    def forward(self, sentences):
        inputs = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sentence_embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        
        return sentence_embeddings


# Example usage
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check if CUDA is available
model = USE().to(device)  # Move the model to the device
batch_size = 1
num_sentences = len(sentence_test)
num_batches = (num_sentences + batch_size - 1) // batch_size  # Calculate the number of batches

embeddings_test = []
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_sentences)
        batch_sentences = sentence_test[start_idx:end_idx]
        batch_sentences = [value for value in batch_sentences if not (isinstance(value, float) and math.isnan(value))]
        if len(batch_sentences)==0:
            batch_sentences.append('Dummy')
        #print(batch_sentences)
        # Tokenize the batch of sentences
        inputs = model.tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Pass the inputs through the model
        outputs = model.model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        
        embeddings_test.append(batch_embeddings)

# Concatenate the batch embeddings into a single tensor
embeddings_test = torch.cat(embeddings_test, dim=0)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
embeddings_test.shape

torch.Size([159156, 768])

In [8]:
len(node_features_test)

159156

In [9]:
len(embeddings_test)

159156

In [10]:
node_features_test = torch.tensor(node_features_test, dtype=torch.float).to(device)
embeddings_test=torch.tensor(embeddings_test, dtype=torch.float).to(device)
feature_test=torch.cat([node_features_test,embeddings_test], dim=1).to(device)
feature_test =F.normalize(feature_test,p=2, dim=-1)

/var/tmp/pbs.3721100.pbshpc/ipykernel_191187/4229892703.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_test=torch.tensor(embeddings_test, dtype=torch.float).to(device)


In [ ]:
class LinkPredictionDataset(Dataset):
    def __init__(self, nodes, node_attributes, edges):
        self.nodes = nodes
        self.node_attributes = node_attributes
        self.edges = edges
        #self.edge_index=edge_index

    def __len__(self):
        return len(self.edges)

    def __getitem__(self, index):
        src, dst,y = self.edges[index]
        # src_node = self.node_attributes[src]
        # dst_node = self.node_attributes[dst]
        #print(src, dst,self.node_attributes,self.y[index])
        return src, dst,self.node_attributes,y
class GCN(nn.Module):
    def __init__(self, in_channels=feature.shape[1], hidden_channels=512, out_channels=128, num_layers=4):
        super().__init__()
        self.conv1 = GraphSAGE(in_channels,out_channels,num_layers)
        #self.conv2 = GraphSAGE(hidden_channels, out_channels,num_layers)
        self.fc1 = Linear(out_channels*2,128)
        #self.fc2=Linear(256,128)
        self.fc3=Linear(128,64)
        self.fc4=Linear(64,16)
        self.fc5=Linear(16,1)
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        #x = self.conv2(x, edge_index).relu()
        #print(x.shape)
        return x

    def decode(self, z,src,dst):
        #return (z[0][src] * z[0][dst]).sum(dim=-1)
        x=torch.cat([z[0][src],z[0][dst]], dim=-1)
        x=F.leaky_relu(self.fc1(x))
        #x=self.fc2(x).relu()
        x=F.leaky_relu(self.fc3(x))
        x=F.leaky_relu(self.fc4(x))
        x=self.fc5(x)
        return x
#edge_index = torch.tensor([train_src,train_dest], dtype=torch.long).to(device)        
model = GCN()
#if torch.cuda.device_count() > 1:model = nn.DataParallel(model)
model.load_state_dict(torch.load('/home/scai/msr/aiy227513/scratch/research/model/model_bert.pth', map_location=device),
                             strict=False)

In [12]:
feature_test

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.0072e-04,
          1.1951e-04, -1.3999e-05],
        [ 0.0000e+00,  1.9977e-03,  0.0000e+00,  ...,  4.9167e-04,
          2.8023e-04, -4.6003e-05],
        [ 0.0000e+00,  4.9641e-04,  0.0000e+00,  ...,  4.3379e-04,
          1.0266e-04, -4.0684e-05],
        ...,
        [ 0.0000e+00,  2.4882e-03,  0.0000e+00,  ...,  3.6538e-04,
          6.5855e-05, -4.2943e-05],
        [ 0.0000e+00,  1.2758e-02,  0.0000e+00,  ...,  1.4996e-04,
          1.3675e-04, -9.9754e-05],
        [ 0.0000e+00,  1.9912e-03,  0.0000e+00,  ...,  3.1863e-04,
          1.4046e-04, -4.0378e-05]], device='cuda:0')

In [13]:
SAMPLE_SIZE = 1000

with torch.no_grad():
    #y = torch.tensor(test_label, dtype=torch.float)
    #edge_index = torch.tensor([test_src,test_dest], dtype=torch.long).to(device)
    test_dataset = LinkPredictionDataset(node_ids,feature,edges)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    recommendations = {}
    for i in feature_test:
        #src_nodes = np.random.randint(0, len(feature), size=SAMPLE_SIZE)
        src_nodes = torch.from_numpy(i).to(device)
        x=random.randint(len(feature), size=(SAMPLE_SIZE))
        #l = np.random.randint(0, len(feature), size=SAMPLE_SIZE)
        for j in x:
                dst_node = torch.from_numpy(feature[j]).to(device)
        #z = model.encode(node_attributes,edge_index)
                z=torch.cat([src_nodes,dst_node], dim=1).to(device)
                predictions=torch.sigmoid(model.decode(z)).view(-1)
        recommendations[dst_node] = {'src_nodes': src_nodes, 'confidence': predictions}
torch.save(recommendations, 'research/citation_recommendation/random_recommendations_ankit.pt')

NameError: name 'edges' is not defined